In [2]:
# -------------------------------------------------------
# 📦 STEP 0: SETUP & DATA UNDERSTANDING
# Project: PhonePe + UPI Payments Analytics
# Author: Adarsh Ajit Singh
# -------------------------------------------------------

import pandas as pd

pd.set_option('display.max_columns', None)

# -------------------------------------------------------
# 📂 Load the Datasets
# -------------------------------------------------------
phonepe_file = "/content/Phonepe-Final-Dataset.xlsx"
upi_file = "/content/UPI-Transactions-Enhanced.csv"

# Load only All_Transactions sheet for initial exploration
phonepe_df = pd.read_excel(phonepe_file, sheet_name="All_Transactions")

# Load UPI dataset
upi_df = pd.read_csv(upi_file)

# -------------------------------------------------------
# 🧭 Quick Exploration
# -------------------------------------------------------
print("✅ Datasets Loaded Successfully\n")

print("📘 PhonePe Shape:", phonepe_df.shape)
print("📘 UPI Shape:", upi_df.shape)

print("\n🔹 PhonePe Columns:")
print(phonepe_df.columns.tolist())

print("\n🔹 UPI Columns:")
print(upi_df.columns.tolist())

print("\n📊 PhonePe Sample:")
display(phonepe_df.head(3))

print("\n📊 UPI Sample:")
display(upi_df.head(3))

print("\n📈 Missing Values in UPI Dataset:")
print(upi_df.isnull().sum().head(10))


✅ Datasets Loaded Successfully

📘 PhonePe Shape: (300000, 8)
📘 UPI Shape: (50000, 24)

🔹 PhonePe Columns:
['Transaction_ID', 'Amount', 'User_ID', 'Service', 'Service Type', 'Payment_Status', 'Reason', 'Date']

🔹 UPI Columns:
['Transaction_ID', 'Transaction_Date', 'Transaction_Time', 'Amount', 'Payment_Channel', 'Partner_Bank', 'Region', 'State', 'City_Tier', 'Device_Type', 'Service_Type', 'Payment_Status', 'Promo_Used', 'Referral_Source', 'Age_Group', 'Gender', 'Occupation', 'Failure_Reason', 'Platform_Fee_%', 'Revenue_Generated', 'PhonePe_Share_%', 'Transaction_Frequency', 'Engagement_Score', 'LTV_Score']

📊 PhonePe Sample:


,Transaction_ID,Amount,User_ID,Service,Service Type,Payment_Status,Reason,Date
0,RCG_0C338474B366,926.59,PP0021371,Recharge_Bills,FASTag Recharge,Successful,Successful,2024-06-09
1,RCG_6B3B86B07A76,1211.64,PP0002388,Recharge_Bills,DTH,Successful,Successful,2024-08-04
2,RCG_767822392A0E,746.27,PP1101831,Recharge_Bills,Cable TV,Successful,Successful,2024-02-19



📊 UPI Sample:


,Transaction_ID,Transaction_Date,Transaction_Time,Amount,Payment_Channel,Partner_Bank,Region,State,City_Tier,Device_Type,Service_Type,Payment_Status,Promo_Used,Referral_Source,Age_Group,Gender,Occupation,Failure_Reason,Platform_Fee_%,Revenue_Generated,PhonePe_Share_%,Transaction_Frequency,Engagement_Score,LTV_Score
0,62aa4f34-7e48-4413-a2c3-91a6d28d31d7,2024-12-12,05:27:07,4663.92,Intent,ICICI Bank,East,Assam,Tier-3,Android,Recharge & Bills,Success,False,Direct,25-34,Male,Self-Employed,NaN,0.30,13.99,35.33,10,62,59
1,cac84924-d8be-425f-8f71-b9a6a756beae,2024-12-01,15:54:18,3084.65,QR Code,SBI,West,Maharashtra,Tier-1,Android,Recharge & Bills,Success,True,Direct,25-34,Female,Salaried,NaN,0.87,26.84,40.55,1,72,47
2,5e649248-51d5-4f4d-a3ba-bdfbd0265301,2024-12-09,18:43:16,30229.39,QR Code,IndusInd Bank,North,Delhi,Tier-2,Android,Recharge & Bills,Success,False,Direct,35-44,Male,Student,NaN,0.45,136.03,41.59,6,98,63



📈 Missing Values in UPI Dataset:
Transaction_ID      0
Transaction_Date    0
Transaction_Time    0
Amount              0
Payment_Channel     0
Partner_Bank        0
Region              0
State               0
City_Tier           0
Device_Type         0
dtype: int64


In [3]:
# -------------------------------------------------------
# 🧩 STEP 1: ETL CLEANING & STANDARDIZATION
# -------------------------------------------------------

import pandas as pd

# Reload datasets (if not already in memory)
phonepe_df = pd.read_excel("/content/Phonepe-Final-Dataset.xlsx", sheet_name="All_Transactions")
upi_df = pd.read_csv("/content/UPI-Transactions-Enhanced.csv")

# -------------------------------------------------------
# 🔹 1. Clean Column Names
# -------------------------------------------------------
phonepe_df.columns = (
    phonepe_df.columns.str.lower().str.strip().str.replace(" ", "_")
)
upi_df.columns = (
    upi_df.columns.str.lower().str.strip().str.replace(" ", "_")
)

In [4]:
# -------------------------------------------------------
# 🔹 2. Align UPI Columns to Match PhonePe
# -------------------------------------------------------
rename_map = {
    "transaction_id": "transaction_id",
    "transaction_date": "date",
    "transaction_amount": "transaction_amount",
    "service_type": "service_type",
    "payment_status": "payment_status",
    "failure_reason": "failure_reason",
    "region": "region",
    "state": "state"
}
upi_df.rename(columns=rename_map, inplace=True)


In [5]:
# -------------------------------------------------------
# 🔹 3. Clean Nulls & Formats
# -------------------------------------------------------
upi_df['date'] = pd.to_datetime(upi_df['date'], errors='coerce')
upi_df['failure_reason'] = upi_df['failure_reason'].fillna('None')
upi_df['payment_status'] = upi_df['payment_status'].str.capitalize()
upi_df.drop_duplicates(subset='transaction_id', inplace=True)


In [8]:
print("🔹 UPI Columns:")
print(list(upi_df.columns))


🔹 UPI Columns:
['transaction_id', 'date', 'transaction_time', 'amount', 'payment_channel', 'partner_bank', 'region', 'state', 'city_tier', 'device_type', 'service_type', 'payment_status', 'promo_used', 'referral_source', 'age_group', 'gender', 'occupation', 'failure_reason', 'platform_fee_%', 'revenue_generated', 'phonepe_share_%', 'transaction_frequency', 'engagement_score', 'ltv_score', 'year', 'month', 'day_of_week', 'platform_fee_percent']


In [9]:
# -------------------------------------------------------
# 🔹 4. Add Derived Fields (Safe Check)
# -------------------------------------------------------

# Identify which column represents transaction amount
amount_col = None
for possible_col in ['transaction_amount', 'amount', 'txn_amount', 'value', 'upi_amount']:
    if possible_col in upi_df.columns:
        amount_col = possible_col
        break

if amount_col:
    upi_df['platform_fee_percent'] = 0.5
    upi_df['revenue_generated'] = upi_df[amount_col] * (upi_df['platform_fee_percent'] / 100)
else:
    print("⚠️ No transaction amount column found — please verify your dataset columns.")


In [10]:
# -------------------------------------------------------
# 🔹 5. Preview Cleaned Data
# -------------------------------------------------------
print("✅ Cleaned UPI Dataset:")
display(upi_df.head(10))

# -------------------------------------------------------
# 🔹 6. Save Locally for BigQuery Upload
# -------------------------------------------------------
upi_df.to_csv("/content/upi_cleaned.csv", index=False)
phonepe_df.to_csv("/content/phonepe_cleaned.csv", index=False)

print("\n✅ ETL complete! Cleaned datasets exported successfully.")

✅ Cleaned UPI Dataset:


,transaction_id,date,transaction_time,amount,payment_channel,partner_bank,region,state,city_tier,device_type,service_type,payment_status,promo_used,referral_source,age_group,gender,occupation,failure_reason,platform_fee_%,revenue_generated,phonepe_share_%,transaction_frequency,engagement_score,ltv_score,year,month,day_of_week,platform_fee_percent
0,62aa4f34-7e48-4413-a2c3-91a6d28d31d7,2024-12-12,05:27:07,4663.92,Intent,ICICI Bank,East,Assam,Tier-3,Android,Recharge & Bills,Success,False,Direct,25-34,Male,Self-Employed,None,0.30,23.31960,35.33,10,62,59,2024,12,Thursday,0.5
1,cac84924-d8be-425f-8f71-b9a6a756beae,2024-12-01,15:54:18,3084.65,QR Code,SBI,West,Maharashtra,Tier-1,Android,Recharge & Bills,Success,True,Direct,25-34,Female,Salaried,None,0.87,15.42325,40.55,1,72,47,2024,12,Sunday,0.5
2,5e649248-51d5-4f4d-a3ba-bdfbd0265301,2024-12-09,18:43:16,30229.39,QR Code,IndusInd Bank,North,Delhi,Tier-2,Android,Recharge & Bills,Success,False,Direct,35-44,Male,Student,None,0.45,151.14695,41.59,6,98,63,2024,12,Monday,0.5
3,ad55835c-4554-4e23-ac95-50eef74c6b75,2024-12-12,11:13:58,48307.51,Bank Transfer,IndusInd Bank,East,Assam,Tier-3,Android,Loans,Success,True,Marketing Campaign,45-54,Female,Student,None,0.51,241.53755,40.94,12,90,87,2024,12,Thursday,0.5
4,ea40fc82-45cc-4512-bf0e-06a1b6b48824,2025-03-15,07:23:46,25160.93,QR Code,Union Bank,East,West Bengal,Tier-2,Android,Insurance,Success,False,Direct,25-34,Female,Salaried,None,0.56,125.80465,40.22,6,92,77,2025,3,Saturday,0.5
5,5fccc408-52db-4012-bb70-a945700da5a0,2025-02-28,20:49:46,2623.19,QR Code,Axis Bank,West,Maharashtra,Tier-3,Android,Insurance,Success,False,Direct,35-44,Male,Salaried,None,0.43,13.11595,35.63,19,99,86,2025,2,Friday,0.5
6,56fcb124-acf2-4619-9c28-5439fde36e9f,2025-03-04,11:56:32,11099.87,Intent,Bank of Baroda,North,Delhi,Tier-1,Android,Money Transfer,Success,False,Direct,45-54,Male,Retired,None,0.22,55.49935,35.05,20,62,43,2025,3,Tuesday,0.5
7,47e4942d-f9ac-487f-aef4-a6e41740fe0b,2025-07-01,19:16:36,12277.41,UPI ID,IndusInd Bank,North,Haryana,Tier-2,iOS,Recharge & Bills,Fail,False,Referral,35-44,Female,Student,Bank Timeout,0.61,61.38705,41.50,12,53,60,2025,7,Tuesday,0.5
8,7ceb5ec3-9e83-49be-92c6-a3239a6c1c21,2024-11-04,19:42:05,16925.96,UPI ID,ICICI Bank,East,Bihar,Tier-2,Android,Loans,Success,False,Direct,35-44,Female,Student,None,0.70,84.62980,42.96,24,92,42,2024,11,Monday,0.5
9,b2431183-5155-443b-ab35-b5d6b3c6a22b,2025-03-31,01:43:01,2227.99,QR Code,Axis Bank,South,Andhra Pradesh,Tier-3,Android,Money Transfer,Success,False,Direct,18-24,Male,Student,None,0.30,11.13995,44.34,20,90,63,2025,3,Monday,0.5



✅ ETL complete! Cleaned datasets exported successfully.
